**NOTE:** A dynamic, web-based version of VRS-Python may be loaded through [mybinder.org](https://mybinder.org/v2/gh/ga4gh/vrs-python/pgxhttps://mybinder.org/v2/gh/ga4gh/vrs-python/pgx) where this notebook may be run without any local installation. Steps:
1. Navigate to the [VRS-Python mybinder.org build](https://mybinder.org/v2/gh/ga4gh/vrs-python/pgx)
2. Once the Binder build is completed, navigate to `notebooks/Inlined and Referenced Objects.ipynb`
3. Under kernel, select `VRS_kernel`

# Inlined and Referenced Objects

## Replacing Identifiable Objects with their Identifiers

As an example, `Allele` contains a `location`, which must be `oneOf` 1) a subclass of `Location`, or 2) a `CURIE`.

Attributes that may be either objects or object identifiers are called _referable attributes_. When a referable attribute contains an object, the object is said to be _inlined_ ; when it is an identifier, it said to be _referenced_. 

`vr-python` provides code to convert referable attributes between their inlined and referenced forms. Replacing an inlined object with a referenced form is provided by `ga4gh_enref`, and replacing references with their inlined form is provided by `ga4gh_deref`.

Dereferencing an object requires having an _object store_ that converts GA4GH identifiers into GA4GH VR objects. In the simplest case, this may be a Python dictionary.  The `vr-python` implementation includes a Redis-backed object store that provides high-performance object retrieval.

![graphic](https://i.imgur.com/vBY3Pu8.png)

In [1]:
from ga4gh.core import ga4gh_identify
from ga4gh.vrs import models, vrs_deref, vrs_enref

import json
def ppo(o):
    """pretty print object as json"""
    print(json.dumps(o.as_dict(), indent=2))

/Users/ahw001/.local/share/virtualenvs/vrs-python-z7zdIrLG/lib/python3.9/site-packages/python_jsonschema_objects/__init__.py:49: UserWarning: Schema version http://json-schema.org/draft-07/schema not recognized. Some keywords and features may not be supported.
  warnings.warn(


In [2]:
# A sample Allele
allele_dict = {
    'location': {
        'interval': {
            'end': 32936732,
            'start': 32936731,
            'type': 'SimpleInterval'},
        'sequence_id': 'ga4gh:SQ._0wi-qoDrvram155UmcSC-zA5ZK4fpLT',
        'type': 'SequenceLocation'},
    'state': {
        'sequence': 'A',
        'type': 'SequenceState'},
    'type': 'Allele'}

In [3]:
allele = models.Allele(**allele_dict)
allele._id = ga4gh_identify(allele)
allele.location._id = ga4gh_identify(allele.location)
ppo(allele)

{
  "_id": "ga4gh:VA.mpIbo0Vv4HT-Oh3g5SWcuzAR2mue3yL-",
  "type": "Allele",
  "location": {
    "_id": "ga4gh:VSL.v9K0mcjQVugxTDIcdi7GBJ_R6fZ1lsYq",
    "type": "SequenceLocation",
    "sequence_id": "ga4gh:SQ._0wi-qoDrvram155UmcSC-zA5ZK4fpLT",
    "interval": {
      "type": "SimpleInterval",
      "start": 32936731,
      "end": 32936732
    }
  },
  "state": {
    "type": "SequenceState",
    "sequence": "A"
  }
}


## ga4gh_enref and ga4gh_deref using a Python dictionary

In [4]:
object_store = {}

In [5]:
allele_refd = vrs_enref(allele, object_store=object_store)
ppo(allele_refd)

{
  "_id": "ga4gh:VA.mpIbo0Vv4HT-Oh3g5SWcuzAR2mue3yL-",
  "type": "Allele",
  "location": "ga4gh:VSL.v9K0mcjQVugxTDIcdi7GBJ_R6fZ1lsYq",
  "state": {
    "type": "SequenceState",
    "sequence": "A"
  }
}


In [14]:
object_store.keys()

dict_keys(['ga4gh:VSL.v9K0mcjQVugxTDIcdi7GBJ_R6fZ1lsYq', 'ga4gh:VA.mpIbo0Vv4HT-Oh3g5SWcuzAR2mue3yL-'])

In [13]:
ppo(object_store["ga4gh:VSL.v9K0mcjQVugxTDIcdi7GBJ_R6fZ1lsYq"])

{
  "_id": "ga4gh:VSL.v9K0mcjQVugxTDIcdi7GBJ_R6fZ1lsYq",
  "type": "SequenceLocation",
  "sequence_id": "ga4gh:SQ._0wi-qoDrvram155UmcSC-zA5ZK4fpLT",
  "interval": {
    "type": "SimpleInterval",
    "start": 32936731,
    "end": 32936732
  }
}


In [8]:
allele_inld = vrs_deref(allele_refd, object_store=object_store)
ppo(allele_inld)

{
  "_id": "ga4gh:VA.mpIbo0Vv4HT-Oh3g5SWcuzAR2mue3yL-",
  "type": "Allele",
  "location": {
    "_id": "ga4gh:VSL.v9K0mcjQVugxTDIcdi7GBJ_R6fZ1lsYq",
    "type": "SequenceLocation",
    "sequence_id": "ga4gh:SQ._0wi-qoDrvram155UmcSC-zA5ZK4fpLT",
    "interval": {
      "type": "SimpleInterval",
      "start": 32936731,
      "end": 32936732
    }
  },
  "state": {
    "type": "SequenceState",
    "sequence": "A"
  }
}
